In [ ]:
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using Revise, Genie, DelimitedFiles, DCAUtils, JLD2, PyPlot

@load "../data_Genie/pars_dbd.jld2"


In [ ]:
using PyPlot, Statistics, LinearAlgebra

# ---------- PARAMETERS ----------
models = [(8,8), (16,4), (16,8), (16,10),
    (32,8), (16,16), (32,16), (64,16), (128,16),
    (100,50)
]

N_comp_values = [2,4,8,16]

base_dir = "../data_msa_autoencoder/debug/"
ste_dir  = "../data_msa_autoencoder/debug_STE/"

# pre-trained fields/couplings
# (assume h_dbd, J_dbd are already loaded in your session)

# ---------- HELPER ----------
function compute_energy_from_fasta(path::String, h, J)
    aln = read_fasta_alignment(path, 0.9)
    return energy(aln, h, J)
end

function pearson(x::Vector{Float64}, y::Vector{Float64})
    xm, ym = mean(x), mean(y)
    num = sum((x .- xm) .* (y .- ym))
    den = sqrt(sum((x .- xm).^2) * sum((y .- ym).^2))
    return num/den
end

E_DCA = compute_energy_from_fasta("../Sample_conv_denseDBD.dat", h_dbd, J_dbd);
E_nat = compute_energy_from_fasta("../data_msa_autoencoder/alignments/DBD_alignment.clean.a2m", h_dbd, J_dbd);

# ---------- ANALYSIS ----------
for (d,q) in models
    figure(figsize=(12,6))
    
    for (i,N) in enumerate(N_comp_values)
        # file paths
        base_path = "$(base_dir)DBD_shuffle_d$(d)_qq$(q)_gmm$(N).00.fasta"
        ste_path  = "$(ste_dir)DBD_shuffle_d$(d)_qq$(q)_gmm$(N).00.fasta"
        
        # compute energies
        E_base = compute_energy_from_fasta(base_path, h_dbd, J_dbd)
        E_ste  = compute_energy_from_fasta(ste_path,  h_dbd, J_dbd)

        # ---- Histograms ----
        subplot(2, length(N_comp_values), i)
        hist(E_base, bins=30, alpha=0.5, label="BASE")
        hist(E_ste, bins=30, alpha=0.5, label="STE")
        hist(E_DCA, bins=30, histtype = "step", color = "grey", alpha=0.5, label="DCA")
        hist(E_nat, bins=30, histtype = "step", color = "black", alpha=0.5, label="Nat") 
        title("d=$d, q=$q, N=$N")
        legend()

        # ---- Scatter with Pearson ----
        subplot(2, length(N_comp_values), length(N_comp_values)+i)
        scatter(E_base, E_ste, alpha=0.5, s=10)
        r = pearson(E_base, E_ste)
        plot([minimum(E_base), maximum(E_base)], [minimum(E_base), maximum(E_base)],
             "k--", lw=1)
        xlabel("BASE energy")
        ylabel("STE energy")
        title("ρ = $(round(r, digits=3))")
    end
    
    tight_layout()
    savefig("../energy_comparison_d$(d)_q$(q).png", dpi=200)
end


In [ ]:
using PyPlot


# ---------- PARAMETERS ----------
models = [(8,8), (16,4), (16,8), (16,10),
    (32,8), (16,16), (32,16), (64,16), (128,16),
    (100,50)
]
N_comp_values = [2,4,8,16]

base_dir = "../data_msa_autoencoder/debug/"
ste_dir  = "../data_msa_autoencoder/debug_STE/"

# assume h_dbd, J_dbd are loaded

# ---------- HELPERS ----------
function compute_energy_from_fasta(path::String, h, J)
    aln = read_fasta_alignment(path, 0.9)
    return energy(aln, h, J)
end

function pearson(x::Vector{Float64}, y::Vector{Float64})
    xm, ym = mean(x), mean(y)
    num = sum((x .- xm) .* (y .- ym))
    den = sqrt(sum((x .- xm).^2) * sum((y .- ym).^2))
    return num/den
end

# ---------- STORAGE ----------
recap = []  # will hold tuples: (d,q,N, mean_base, mean_ste, corr)

for (d,q) in models
    for N in N_comp_values
        base_path = "$(base_dir)DBD_shuffle_d$(d)_qq$(q)_gmm$(N).00.fasta"
        ste_path  = "$(ste_dir)DBD_shuffle_d$(d)_qq$(q)_gmm$(N).00.fasta"

        E_base = compute_energy_from_fasta(base_path, h_dbd, J_dbd)
        E_ste  = compute_energy_from_fasta(ste_path,  h_dbd, J_dbd)

        mean_base = mean(E_base)
        mean_ste  = mean(E_ste)
        r = pearson(E_base, E_ste)

        push!(recap, (d,q,N,mean_base,mean_ste,r))
    end
end


using PyPlot

# ---------- PLOT ----------
figure(figsize=(12,5))

# assign a color to each (d,q) pair
models = unique([(x[1],x[2]) for x in recap])
colors = Dict(model => get_cmap("tab20")(i/length(models))
              for (i,model) in enumerate(models))

# === 1) Mean energies: BASE vs STE ===
subplot(1,2,1)

for (d,q,N,mean_base,mean_ste,r) in recap
    color = colors[(d,q)]
    scatter(mean_base, mean_ste, s=40, color=color)
    text(mean_base, mean_ste, string(N),
         fontsize=8, ha="left", va="bottom", color=color)
end

# add diagonal reference line
xlims = (minimum(x[4] for x in recap), maximum(x[4] for x in recap))
plot(xlims, xlims, "k--", lw=1)
xlabel("Mean BASE energy")
ylabel("Mean STE energy")
title("Mean energies: BASE vs STE")

# === 2) Correlation vs N_comp ===
subplot(1,2,2)

for (d,q) in models
    Ns   = [x[3] for x in recap if x[1]==d && x[2]==q]
    cors = [x[6] for x in recap if x[1]==d && x[2]==q]
    plot(Ns, cors, marker="o", color=colors[(d,q)], label="d=$d, q=$q")
end

xlabel("N_comp")
ylabel("Pearson correlation")
title("Correlation of energies")
legend(bbox_to_anchor=(1.05,1), loc="upper left")

tight_layout()
savefig("../recap_energy_summary.png", dpi=200)
